In [3]:
from pynwb.spec import NWBDatasetSpec, NWBNamespaceBuilder, NWBGroupSpec
from pynwb.spec import NWBNamespaceBuilder, export_spec, NWBGroupSpec, NWBAttributeSpec
from pynwb.spec import NWBDatasetSpec


ns_path = "mylab.namespace.yaml"
ext_source = "mylab.extensions.yaml"

ns_builder = NWBNamespaceBuilder('Extension for use in my Lab', "mylab", version='0.1.0')
# Now we define the data structures. We use `NWBDataInterface` as the base type,
# which is the most primitive type you are likely to use as a base. The name of the
# class is `CorticalSurface`, and it requires two matrices, `vertices` and
# `faces`.
ns_builder.include_type('NWBDataInterface', namespace='core')
ns_builder.include_type('DynamicTable', namespace='core')
ns_builder.include_type('VectorData', namespace='core')
ns_builder.include_type('VectorIndex', namespace='core')

timestamps = NWBDatasetSpec(
        name='timestamps',
        dtype='float32',
        dims=['num_events'],
        shape=[None],
        doc=('Event timestamps, in seconds, relative to the common experiment master-clock stored in '
             'NWBFile.timestamps_reference_time.'),
        attributes=[
            NWBAttributeSpec(
                name='unit',
                dtype='text',
                value='seconds',
                doc="Unit of measurement for timestamps, which is fixed to 'seconds'.",
            ),
            NWBAttributeSpec(
                name='resolution',
                dtype='float32',
                doc=('The smallest possible difference between two event times. Usually 1 divided by the event time '
                     'sampling rate on the data acquisition system.'),
                required=False,
            )
        ]
    )

events = NWBGroupSpec(
    neurodata_type_def='Events',
    neurodata_type_inc='NWBDataInterface',
    doc='A list of timestamps, stored in seconds, of an event.',
    attributes=[
        NWBAttributeSpec(
            name='description',
            dtype='text',
            doc='Description of the event.',
        ),
    ],
    datasets=[timestamps]
)

ns_builder.add_spec(ext_source, events)
ns_builder.export(ns_path)

In [6]:
from pynwb import get_class, load_namespaces
from pynwb import register_class, load_namespaces
from pynwb.core import NWBContainer, NWBDataInterface
from pynwb.base import TimeSeries
from collections.abc import Iterable
from pynwb.device import  Device
from hdmf.utils import docval, popargs, get_docval, popargs_to_dict
from pynwb.core import DynamicTable, VectorData
import numpy as np
from hdmf.utils import docval, getargs, popargs, popargs_to_dict, get_docval, call_docval_func
import warnings

from pynwb.io.core import NWBContainerMapper
from pynwb import register_map

from hdmf.build import LinkBuilder


ns_path = "mylab.namespace.yaml"
load_namespaces(ns_path)

@register_class('Events', 'mylab')
class Events(NWBDataInterface):
    """
    A list of timestamps, stored in seconds, of an event.
    """

    __nwbfields__ = ('description',
                     'timestamps',
                     'timestampsResolution',
                     {'name': 'unit', 'settable': False})

    @docval({'name': 'name', 'type': str, 'doc': 'The name of this Events object'},  # required
            {'name': 'description', 'type': str, 'doc': 'The name of this Events object'},  # required
            {'name': 'timestamps', 'type': ('array_data', 'data'),  # required
             'doc': ('Event timestamps, in seconds, relative to the common experiment master-clock '
                     'stored in NWBFile.timestamps_reference_time.'),
             'shape': (None,)},
            {'name': 'resolution', 'type': float,
             'doc': ('The smallest possible difference between two event times. Usually 1 divided '
                     'by the event time sampling rate on the data acquisition system.'),
             'default': None})
    def __init__(self, **kwargs):
        description, timestamps, resolution = popargs('description', 'timestamps', 'resolution', kwargs)
        # call_docval_func(super().__init__, kwargs)

        super(Events, self).__init__(name=kwargs['name'])
        self.description = description
        self.timestamps = timestamps
        self.resolution = resolution
        self.fields['unit'] = 'seconds'

@register_map(Events)
class EventsMap(NWBContainerMapper):

    def __init__(self, spec):
        super(EventsMap, self).__init__(spec)
        data_spec = self.spec.get_dataset('timestamps')
        self.map_spec('unit', data_spec.get_attribute('unit'))
        self.map_spec('timestampsResolution', data_spec.get_attribute('resolution'))

        # TODO map the sync group to something
        # sync_spec = self.spec.get_group('sync')
        # self.unmap(sync_spec)

In [7]:

from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import load_namespaces

from pynwb import NWBFile

nwbfile = NWBFile(
    'my first synthetic recording',
    'EXAMPLE_ID',
    datetime.now(tzlocal()),
)

events = Events(
            name='Events',
            description='events from my experiment',
            timestamps=[0., 1., 2.],
            resolution=1e-5
        )

from pynwb import NWBHDF5IO


nwbfile.add_acquisition(events)

with NWBHDF5IO("basics_tutorial.nwb", "w") as io:
    io.write(nwbfile)

with NWBHDF5IO("basics_tutorial.nwb", "r", load_namespaces=True) as io:
    read_nwbfile = io.read()


print('')

In [20]:
io.close()

In [ ]:
with NWBHDF5IO("basics_tutorial.nwb", "r", load_namespaces=True) as io:
    read_nwbfile = io.read()
